In [2]:
# Install dependencies if running in Colab
!pip install transformers datasets torch evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import evaluate

In [4]:
# 1. Load dataset (IMDb: 50k movie reviews labeled positive/negative)
dataset = load_dataset("imdb")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# 2. Load pre-trained tokenizer and model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [6]:
# 3. Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Use smaller subset for faster training/testing
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(500))

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
# 4. Aaccuracy
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [8]:
# 5. Training setup
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to='none'
)

In [9]:
# 6. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-2524890946.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
# 7. Train
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.308800,0.432362,0.812000
2,0.109800,0.694718,0.832000


TrainOutput(global_step=250, training_loss=0.33433748626708987, metrics={'train_runtime': 34.612, 'train_samples_per_second': 57.783, 'train_steps_per_second': 7.223, 'total_flos': 66233699328000.0, 'train_loss': 0.33433748626708987, 'epoch': 2.0})

In [11]:
# 8. Evaluate
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.6947178840637207, 'eval_accuracy': 0.832, 'eval_runtime': 1.6483, 'eval_samples_per_second': 303.349, 'eval_steps_per_second': 38.222, 'epoch': 2.0}


In [15]:
# 9. Test on custom text
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

print(sentiment_pipeline("I absolutely loved this movie!"))
print(sentiment_pipeline("The movie was terrible and boring."))

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9997151494026184}]
[{'label': 'NEGATIVE', 'score': 0.9997716546058655}]
